In [3]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from astropy.cosmology import Planck18 as cosmo

In [ ]:
CRIT_DENS_TRANSITION = 1.5
Deltac = 1.68
NSFR_low = 250
NSFR_high = 200
MIN_DENSITY_LOW_LIMIT = 9e-8

In [4]:
dir(cosmo)

['H',
 'H0',
 'Neff',
 'Ob',
 'Ob0',
 'Ode',
 'Ode0',
 'Odm',
 'Odm0',
 'Ogamma',
 'Ogamma0',
 'Ok',
 'Ok0',
 'Om',
 'Om0',
 'Onu',
 'Onu0',
 'Otot',
 'Otot0',
 'Tcmb',
 'Tcmb0',
 'Tnu',
 'Tnu0',
 '_EdS_age',
 '_EdS_comoving_distance_z1z2',
 '_EdS_lookback_time',
 '_H0',
 '_Neff',
 '_Ob0',
 '_Ode0',
 '_Odm0',
 '_Ogamma0',
 '_Ok0',
 '_Om0',
 '_Onu0',
 '_T_hypergeometric',
 '_Tcmb0',
 '_Tnu0',
 '__abstractmethods__',
 '__all_parameters__',
 '__annotations__',
 '__astropy_table__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__equiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonflatclass__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_abs_distance_integrand_scalar',
 '_age',
 '_comoving_dista

In [6]:
def dicke(z):
    OMm = cosmo.Om0  # Matter density parameter
    OMl = cosmo.Ode0  # Dark Energy density parameter
    OMr = cosmo.Ogamma0  # Radiation density parameter

    omegaM_z = OMm * (1 + z)**3 / (OMl + OMm * (1 + z)**3 + OMr * (1 + z)**4)
    dick_z = 2.5 * omegaM_z / ( 1/70 + omegaM_z * (209 - omegaM_z) / 140 + omegaM_z**(4/7) )
    dick_0 = 2.5 * OMm      / ( 1/70 + OMm      * (209 - OMm)      / 140 + OMm**(4/7)      )
    return dick_z / (dick_0 * (1.0 + z))

In [ ]:
def initialise_SFRD_Conditional_table(
    int Nfilter, float min_density[], float max_density[], float growthf[], float R[],
    float MassTurnover, float Alpha_star, float Fstar10, bool FAST_FCOLL_TABLES
):

    overdense_large_high = Deltac
    overdense_large_low = CRIT_DENS_TRANSITION

    ln_10 = np.log(10);

    Mmin = MassTurnover/50.
    Mmax = RtoM(R[Nfilter-1])
    Mlim_Fstar = Mass_limit_bisection(Mmin, Mmax, Alpha_star, Fstar10)
    Mmin = np.log(Mmin)

    for i in rnage(NSFR_high):
        overdense_high_table[i] = overdense_large_low + i/(NSFR_high-1)*(overdense_large_high - overdense_large_low)

    for j in range(Nfilter):

        Mmax = RtoM(R[j])
        initialiseGL_Nion_Xray(NGL_SFR, MassTurnover/50., Mmax)

        Mmax = np.log(Mmax)
        MassBin = np.floor( ( Mmax - MinMass )*inv_mass_bin_width ).astype(int)

        MassBinLow = MinMass + mass_bin_width*MassBin

        sigma2 = Sigma_InterpTable[MassBin] + ( Mmax - MassBinLow )*( Sigma_InterpTable[MassBin+1] - Sigma_InterpTable[MassBin] )*inv_mass_bin_width

        if min_density[j]*growthf[j] < -1.:
            overdense_small_low = -1. + MIN_DENSITY_LOW_LIMIT
        else:
            overdense_small_low = min_density[j]*growthf[j]
        overdense_small_high = max_density[j]*growthf[j]
        if overdense_small_high > CRIT_DENS_TRANSITION:
            overdense_small_high = CRIT_DENS_TRANSITION

        for i in range(NSFR_low):
            overdense_val = np.log10(1. + overdense_small_low) + i/(NSFR_low-1)*(np.log10(1.+overdense_small_high)-np.log10(1.+overdense_small_low))
            overdense_low_table[i] = pow(10.,overdense_val)

        for i in range(NSFR_low):
            log10_SFRD_z_low_table[j][i] = GaussLegendreQuad_Nion(1,NGL_SFR,growthf[j],Mmax,sigma2,Deltac,overdense_low_table[i]-1.,MassTurnover,Alpha_star,0.,Fstar10,1.,Mlim_Fstar,0., FAST_FCOLL_TABLES)
            if np.abs(log10_SFRD_z_low_table[j][i]) < 1e-38:
                log10_SFRD_z_low_table[j][i] = 1e-38
            log10_SFRD_z_low_table[j][i] = np.log10(log10_SFRD_z_low_table[j][i])

            log10_SFRD_z_low_table[j][i] += 10.0
            log10_SFRD_z_low_table[j][i] *= ln_10

        for i in range(NSFR_high):
            SFRD_z_high_table[j][i] = Nion_ConditionalM(growthf[j],Mmin,Mmax,sigma2,Deltac,overdense_high_table[i],MassTurnover,Alpha_star,0.,Fstar10,1.,Mlim_Fstar,0., FAST_FCOLL_TABLES);
            SFRD_z_high_table[j][i] *= pow(10., 10.);

In [ ]:
log10_SFRD_z_low_table = ...
SFRD_z_high_table = ...

min_densities[R_ct] = np.min(curr_dens_box)
max_densities[R_ct] = np.max(curr_dens_box)
zpp_growth[R_ct] = dicke(zpp) # zpp is z of donor shell

In [ ]:
fcoll_interp_high_min = CRIT_DENS_TRANSITION
fcoll_interp_high_bin_width = 1 / (NSFR_high-1)*(Deltac - fcoll_interp_high_min)
fcoll_interp_high_bin_width_inv = 1 / fcoll_interp_high_bin_width

def fcoll_interp_min(R_ct):
    if min_densities[R_ct]*zpp_growth[R_ct] <= -1.:
        fcoll_interp_min = np.log10(MIN_DENSITY_LOW_LIMIT)
    else:
        fcoll_interp_min = np.log10(1+min_densities[R_ct]*zpp_growth[R_ct])

def fcoll_interp_bin_width(R_ct):
    if max_densities[R_ct]*zpp_growth[R_ct] > CRIT_DENS_TRANSITION:
        fcoll_interp_bin_width = 1./(NSFR_low-1.)*(np.log10(1+CRIT_DENS_TRANSITION)-fcoll_interp_min(R_ct))
    else:
        fcoll_interp_bin_width = 1./(NSFR_low-1.)*(np.log10(1+max_densities[R_ct]*zpp_growth[R_ct])-fcoll_interp_min(R_ct))

def fcoll_interp_bin_width_inv(R_ct):
    return 1 / fcoll_interp_bin_width(R_ct)

In [ ]:
def fcoll(curr_dens, R_ct):
    if curr_dens < CRIT_DENS_TRANSITION:
        if curr_dens <= -1.:
            fcoll = 0.
        else:
            dens_val = (np.log10(1+curr_dens) - fcoll_interp_min(R_ct)) * fcoll_interp_bin_width_inv(R_ct)
            fcoll_int = np.floor(dens_val).astype(int)

            if fcoll_int < 0 or (fcoll_int + 1) > (NSFR_low - 1):
                if fcoll_int == (NSFR_low - 1):
                    if np.abs(curr_dens - CRIT_DENS_TRANSITION) < 1e-4:
                        # There can be instances where the numerical rounding causes it to go in here,
                        # rather than the curr_dens > global_params.CRIT_DENS_TRANSITION case
                        # This checks for this, and calculates f_coll in this instance, rather than causing it to error
                        dens_val = (curr_dens - fcoll_interp_high_min) * fcoll_interp_high_bin_width_inv
                        fcoll_int = np.floor(dens_val).astype(int)
                        fcoll = (
                            SFRD_z_high_table[R_ct][fcoll_int] * (1 + fcoll_int - dens_val) +
                            SFRD_z_high_table[R_ct][fcoll_int+1] * (dens_val - fcoll_int)
                        )
                    else:
                        fcoll = np.exp(log10_SFRD_z_low_table[R_ct][fcoll_int])
                else:
                    raise ValueError("fcoll_int_boundexceeded_threaded[] = 1")
            else:
                fcoll = np.exp(
                    log10_SFRD_z_low_table[R_ct][fcoll_int] * (1 + fcoll_int - dens_val) +
                    log10_SFRD_z_low_table[R_ct][fcoll_int + 1] * (dens_val - fcoll_int)
                )
    else:
        if curr_dens < 0.99 * Deltac:
            dens_val = (curr_dens - fcoll_interp_high_min) * fcoll_interp_high_bin_width_inv
            fcoll_int = int(np.floor(dens_val))

            if fcoll_int < 0 or (fcoll_int + 1) > (NSFR_high - 1):
                raise ValueError("fcoll_int_boundexceeded_threaded[] = 1")

            fcoll = (
                SFRD_z_high_table[R_ct][fcoll_int] * (1 + fcoll_int - dens_val) +
                SFRD_z_high_table[R_ct][fcoll_int + 1] * (dens_val - fcoll_int)
            )
        else:
            fcoll = 1e10

    return fcoll

def del_fcoll_Rct(curr_dens, R_ct):
    return (1+curr_dens) * fcoll(curr_dens, R_ct)